In [1]:
import numpy as np
import random
import sys
import matplotlib.pyplot as plt
import pandas as pd
import math
import time
a = sys.path.append('../modules/') # endereco das funcoes implementadas por voce!
import plot_3D, auxiliars, salve_doc, sphere, sample_random, Operators_array, Operators_alpina
#import test_operators_06_10_2020 as top
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Poly3DCollection, Line3DCollection

In [2]:
population = {'xmax': 10.0,
                'xmin': 0.0,
                'ymax': 10.0,
                'ymin': 0.0,
                'zlim': 10.0,
                'z_min': 0.0,
                'n_dip': 1,
                'n_pop': 10,
                'homogeneo': True
                }

filhos_mut = {'xmax': 10.0,
                'xmin': 0.0,
                'ymax': 10.0,
                'ymin': 0.0,
                'zlim': 100.0,
                'z_min': 0.0,
                'n': 1,
                'homogeneo': True
                }



In [3]:
pop_inicial = Operators_alpina.create_population(**population)
print(pop_inicial)

[array([[2.94797896, 4.93302884]]), array([[2.26601896, 2.92450826]]), array([[2.99390604, 9.33270514]]), array([[8.6380547, 3.5994388]]), array([[7.20991982, 3.48483309]]), array([[6.30390954, 0.54935509]]), array([[3.74993999, 2.58385768]]), array([[2.38437082, 8.40754512]]), array([[6.89405084, 1.1805787 ]]), array([[1.16013555, 8.78235447]])]


In [21]:
def uniform_crossover(pop_inicial):
    p_shape = tuple(pop_inicial[0].shape) #tuple(mae1.shape)
    filhos = []
    n_filhos = int(len(pop_inicial) / 2)
    pai = pop_inicial[0:n_filhos]
    mae = pop_inicial[n_filhos:len(pop_inicial)]
    print(mae)
    for k in range(n_filhos):
        probs = np.random.rand(p_shape[0], p_shape[1])
        i_pai = pai[k]
        i_mae = mae[k]
        m_mae = np.zeros(p_shape)
        m_pai = np.zeros(p_shape)
        for i in range(p_shape[0]):
            for j in range(p_shape[1]):
                if probs[i,j] < 0.5:
                    m_pai[i,j], m_mae[i,j] = i_mae[i,j], i_pai[i,j]
                else:
                    m_pai[i, j], m_mae[i, j] = i_pai[i, j], i_mae[i, j]
        filhos +=[m_pai, m_mae]
    return filhos

In [23]:
def single_p_crossover(pop_inicial, X = 1):
    filhos = []
    n_filhos = int(len(pop_inicial) / 2)
    pai = np.array(pop_inicial[0:n_filhos])
    inv_pai = pai[::-1]
    mae = np.array(pop_inicial[n_filhos:len(pop_inicial)])
    for i in range(n_filhos):
        pai_new = np.zeros((1,2))
        mae_new = np.zeros((1,2))
        invpai_new = np.zeros((1,2))
        pai_new[0,0], pai_new[0,1] = pai[i][0,:X], mae[i][0,X:]
        mae_new[0,0], mae_new[0,1] = mae[i][0,:X], pai[i][0,X:]
        invpai_new[0,0], invpai_new[0,1] = inv_pai[i][0,:X], mae[i][0,X:]
        filhos +=[pai_new, mae_new, invpai_new]
    return filhos

In [6]:
def crossover_mix_doubles(pais_torneio, escolhidos, fit):
    filhos = []
    n_filhos = int(len(pais_torneio) / 2)
    n1_4_filhos = int(len(pais_torneio)/4)
    pai = np.array(pais_torneio[0:n_filhos])
    inv_pai = pai[::-1]
    mae = np.array(pais_torneio[n_filhos:len(pais_torneio)])
    m_pai = np.array(pais_torneio[n1_4_filhos:n_filhos])
    m_mae = np.array(pais_torneio[n_filhos:(n_filhos+n1_4_filhos)])
    m_pai = np.concatenate((m_pai, m_mae), axis=0)
    # Sorteio das probabilidades de forma randômica.
    probs = np.random.rand(4)

    for j in range(n_filhos):
        num0 = (probs[0] * pai[j] + probs[1] * mae[j])
        filho0 = num0 / (probs[0] + probs[1]) # Cálculo do filho
        #print('Filho=', j, filho)
        num1 = (probs[2] * pai[j] + probs[3] * mae[j])
        filho1 = num1 / (probs[2] + probs[3]) # Cálculo do filho
        
        num2 = (probs[0] * inv_pai[j] + probs[1] * mae[j])
        filho2 = num2 / (probs[0] + probs[1])
        
        num3 = (probs[2] * inv_pai[j] + probs[3] * mae[j])
        filho3 = num3 / (probs[2] + probs[3])
        
        num4 = (probs[0] * pai[j] + probs[1] * m_pai[j])
        filho4 = num4 / (probs[0] + probs[1])
        
        num5 = (probs[2] * pai[j] + probs[3] * m_pai[j])
        filho5 = num5 / (probs[2] + probs[3])
        
        num6 = (probs[0] * mae[j] + probs[1] * m_pai[j])
        filho6 = num6 / (probs[0] + probs[1])
        
        num7 = (probs[2] * mae[j] + probs[3] * m_pai[j])
        filho7 = num7 / (probs[2] + probs[3])
        
        
        filhos += [filho0, filho1, filho2, filho3, filho4, filho5, filho6, filho7]

    return filhos

In [7]:
mae1 = np.random.rand(2,2)
print(mae1)
print(type(mae1))
print('-------------')
pai1 = np.random.rand(2,2)
print(pai1)

[[0.07790816 0.9753555 ]
 [0.07185181 0.76761828]]
<class 'numpy.ndarray'>
-------------
[[0.54134434 0.03121027]
 [0.62761335 0.35597305]]


In [8]:
p_shape = tuple(pop_inicial[0].shape) #tuple(mae1.shape)
filhos = []
n_filhos = int(len(pop_inicial) / 2)
pai = pop_inicial[0:n_filhos]
mae = pop_inicial[n_filhos:len(pop_inicial)]
for k in range(n_filhos):
    probs = np.random.rand(p_shape[0], p_shape[1])
    i_pai = pai[k]
    i_mae = mae[k]
    m_mae = np.zeros(p_shape)
    m_pai = np.zeros(p_shape)
    for i in range(p_shape[0]):
        for j in range(p_shape[1]):
            if probs[i,j] < 0.5:
                m_pai[i,j], m_mae[i,j] = i_mae[i,j], i_pai[i,j]
            else:
                m_pai[i, j], m_mae[i, j] = i_pai[i, j], i_mae[i, j]
    filhos +=[m_pai, m_mae]
print('------------------------------')
print(probs)


------------------------------
[[0.57310295 0.00578259]]


In [9]:
print(pai)
print('-----------------')
print(mae)

[array([[2.94797896, 4.93302884]]), array([[2.26601896, 2.92450826]]), array([[2.99390604, 9.33270514]]), array([[8.6380547, 3.5994388]]), array([[7.20991982, 3.48483309]])]
-----------------
[array([[6.30390954, 0.54935509]]), array([[3.74993999, 2.58385768]]), array([[2.38437082, 8.40754512]]), array([[6.89405084, 1.1805787 ]]), array([[1.16013555, 8.78235447]])]


In [10]:
print(filhos)
print(len(filhos))

[array([[6.30390954, 0.54935509]]), array([[2.94797896, 4.93302884]]), array([[2.26601896, 2.92450826]]), array([[3.74993999, 2.58385768]]), array([[2.99390604, 9.33270514]]), array([[2.38437082, 8.40754512]]), array([[8.6380547, 1.1805787]]), array([[6.89405084, 3.5994388 ]]), array([[7.20991982, 8.78235447]]), array([[1.16013555, 3.48483309]])]
10


In [11]:
print(p_shape)
print(probs)
print(len(probs))

(1, 2)
[[0.57310295 0.00578259]]
1


In [12]:
print(pai)
print('---------------------------')
print(mae)

[array([[2.94797896, 4.93302884]]), array([[2.26601896, 2.92450826]]), array([[2.99390604, 9.33270514]]), array([[8.6380547, 3.5994388]]), array([[7.20991982, 3.48483309]])]
---------------------------
[array([[6.30390954, 0.54935509]]), array([[3.74993999, 2.58385768]]), array([[2.38437082, 8.40754512]]), array([[6.89405084, 1.1805787 ]]), array([[1.16013555, 8.78235447]])]


In [13]:
print(filhos)
print(len(filhos))

[array([[6.30390954, 0.54935509]]), array([[2.94797896, 4.93302884]]), array([[2.26601896, 2.92450826]]), array([[3.74993999, 2.58385768]]), array([[2.99390604, 9.33270514]]), array([[2.38437082, 8.40754512]]), array([[8.6380547, 1.1805787]]), array([[6.89405084, 3.5994388 ]]), array([[7.20991982, 8.78235447]]), array([[1.16013555, 3.48483309]])]
10


In [14]:
print(filhos[0])

[[6.30390954 0.54935509]]


In [24]:
test_filho = single_p_crossover(pop_inicial, X = 1)
print(test_filho)
print(len(test_filho))
print(len(test_filho[0]))

[array([[2.94797896, 0.54935509]]), array([[6.30390954, 4.93302884]]), array([[7.20991982, 0.54935509]]), array([[2.26601896, 2.58385768]]), array([[3.74993999, 2.92450826]]), array([[8.6380547 , 2.58385768]]), array([[2.99390604, 8.40754512]]), array([[2.38437082, 9.33270514]]), array([[2.99390604, 8.40754512]]), array([[8.6380547, 1.1805787]]), array([[6.89405084, 3.5994388 ]]), array([[2.26601896, 1.1805787 ]]), array([[7.20991982, 8.78235447]]), array([[1.16013555, 3.48483309]]), array([[2.94797896, 8.78235447]])]
15
1


In [16]:
print(test_filho[0])
print(type(test_filho))

[[2.94797896 4.93302884]]
<class 'list'>


In [17]:
param_select = random.randint(0, 1)
param_select

0

In [18]:
t_t = Operators_array.crossover_polyamory(pop_inicial, 1, 1)

In [19]:
print(t_t)
print(type(t_t))

[array([[4.65717803, 2.70039294]]), array([[4.91607135, 2.36221435]]), array([[6.74848241, 1.98977348]]), array([[6.67858823, 1.76331598]]), array([[6.82205055, 2.22813464]]), array([[6.77920749, 2.08932291]]), array([[6.68691343, 1.79028968]]), array([[6.77078792, 2.06204347]]), array([[4.6443755, 2.7171162]]), array([[4.38467607, 3.05634777]]), array([[3.02179039, 2.75101252]]), array([[3.1362675 , 2.72473303]]), array([[6.14850347, 3.08219619]]), array([[5.77140977, 3.00384916]]), array([[6.54541897, 3.16466149]]), array([[6.31427171, 3.11663709]]), array([[5.81632599, 3.0131812 ]]), array([[6.2688464 , 3.10719928]]), array([[3.01612938, 2.75231207]]), array([[2.90129582, 2.77867339]]), array([[2.68346546, 8.86151463]]), array([[2.63644286, 8.79014314]]), array([[2.68346546, 8.86151463]]), array([[2.63644286, 8.79014314]]), array([[2.73295979, 8.93663773]]), array([[2.70413633, 8.89288913]]), array([[2.64204379, 8.79864431]]), array([[2.69847191, 8.88429161]]), array([[2.68579078, 8

In [20]:
test_filho1 = uniform_crossover(pop_inicial)
print(test_filho1)
print(len(test_filho1))
print(len(test_filho1[0]))

[array([[2.94797896, 0.54935509]]), array([[6.30390954, 4.93302884]]), array([[3.74993999, 2.58385768]]), array([[2.26601896, 2.92450826]]), array([[2.38437082, 8.40754512]]), array([[2.99390604, 9.33270514]]), array([[6.89405084, 3.5994388 ]]), array([[8.6380547, 1.1805787]]), array([[1.16013555, 8.78235447]]), array([[7.20991982, 3.48483309]])]
10
1
